In [1]:
import torch

In [2]:
words = open("../names.txt", "r").read().splitlines()

In [3]:
for word in words[:1]:
    chs = ["."] + list(word) + ["."]
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        print(ch1, ch2, ch3)

. e m
e m m
m m a
m a .


In [4]:
chars = ['.'] + sorted(list(set(''.join(words))))
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for i, s in enumerate(chars)}

In [5]:
N = torch.zeros((27, 27, 27), dtype=torch.int32)

In [6]:
for word in words:
    chs = ["."] + list(word) + ["."]
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ch1 = stoi[ch1]
        ch2 = stoi[ch2]
        ch3 = stoi[ch3]

        N[ch1, ch2, ch3] += 1

In [7]:
# 0, 0, :
# :, 0, 0
# :, 0, :

In [8]:
# Calculate probability distribution

P = (N+1).float()
P /= P.sum(axis=2, keepdim=True)

In [9]:
 P.sum(axis=2, keepdim=True).shape

torch.Size([27, 27, 1])

In [10]:
init = P[0].flatten()
init /= init.sum()

In [11]:
g = torch.Generator().manual_seed(2147483647)

for _ in range(10):
    ix1 = 0; ix2 = torch.multinomial(init, 1, replacement=True, generator=g).item() % 27 # inital bigram
    out = [itos[ix2]]
    while True:
    
        temp = ix1
        ix1 = ix2
        ix2 = torch.multinomial(P[temp, ix2], num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix2])
        
        if ix2 == 0:
            break
    print("".join(out))


aimittain.
umtai.
aside.
alysuzusfxx.
ika.
bryuvinqeven.
hazee.
yen.
aliemmy.
lexane.


In [12]:
# Check the probability of trigram
trig = "emm"
P[*[stoi[s] for s in trig]]

tensor(0.1269)

In [13]:
# calculate the loss
log_likelyhood = 0
num = 0

for word in words:
    chs = ["."] + list(word) + ["."]
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ch1 = stoi[ch1]
        ch2 = stoi[ch2]
        ch3 = stoi[ch3]

        log_likelyhood += torch.log(P[ch1, ch2, ch3]) 
        num += 1

nll = -log_likelyhood / num

In [14]:
nll

tensor(2.6309)

## using neural nets

In [146]:
import random

In [163]:
xs = []
ys = []

training_set = words[:80*len(words) // 100]
dev_set = words[80*len(words) // 100 : 90 * len(words) // 100]
test_set = words[90 * len(words) // 100:]

for word in training_set:
    chs = ["."] + list(word) + ["."]
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ch1 = stoi[ch1]
        ch2 = stoi[ch2]
        ch3 = stoi[ch3]

        xs.append((ch1, ch2))
        ys.append(ch3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = ys.nelement()

In [17]:
import torch.nn.functional as F

In [98]:
g = torch.Generator().manual_seed(2147483647)
w = torch.randn((27, 27, 27), generator=g, requires_grad=True)

In [165]:
# forward pass
logits = torch.stack([w[*x] for x in xs])

# softmax
counts = logits.exp()
probs = counts / counts.sum(axis=1, keepdim=True)

loss = -probs[torch.arange(num), ys].log().mean()

# backward pass
w.grad = None
loss.backward()

# update parameters
w.data -= 100 * w.grad
    
print(f"{loss.data.item():.4f}")

2.2910


In [143]:
# one_hot generates (len(input), num_classes) shape tensor ?
# forward pass
#logits = w[xs]

# softmax
logits = torch.stack([w[*x] for x in xs])

# softmax
counts = logits.exp()
probs = counts / counts.sum(axis=1, keepdim=True)

probs.shape

torch.Size([196113, 27])

In [ ]:
print(x_enc.shape)
print(w.shape)

In [88]:
g = torch.Generator().manual_seed(2147483647)

counts = logits.exp()
probs = counts / counts.sum(axis=1, keepdim=True)

for _ in range(10):
    ix1 = 0; ix2 = torch.multinomial(init, 1, replacement=True, generator=g).item() % 27 # inital bigram
    out = [itos[ix2]]
    while True:
    
        temp = ix1
        ix1 = ix2

        logits = w[(temp, ix2)]
        ix2 = torch.multinomial(probs[temp, ix2], num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix2])
        
        if ix2 == 0:
            break
    print("".join(out))

RuntimeError: prob_dist must be 1 or 2 dim

In [19]:
(w[xs]).shape

torch.Size([196113, 27])